# Introduction: Partitioning Data  

Problem: we have a large dataset that we want to partition into smaller sections in order to run a feature engineering pipeline over these partitions in parallel.
Approach: divide the data into partitions by hashing the customer id to an integer and then modolu (integer) dividing by the number of partitions.

After determining the partition for each customer id, we can iterate over the data and write the partitions to the correct directory. At the end of the process, we'll have `N_PARTITIONS` of customer data, each containing all the data for a subset of customers. 

In [1]:
import pandas as pd
import hashlib

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

N_PARTITIONS = 1000

## Hashing

Hashing is the process of mapping a string of characters to a fixed length value or key. For this problem, we'll take a string, the customer id (`msno` in the data) and map it to an integer (16 bits). To convert to a partition number, we then integer (modulo) divide this integer by the number of partitions.

### Consistency of Hashing

To make sure that the integer for a given string is always the same, we use the `md5` algorithm form the `hashlib` library. There is no way to go backwards from an integer to the string, but we can always go from the string to the same exact hash. Since we don't need to partition our customers in any particular groupings, randomly assigning customers to partitions is an acceptable approach.

In [2]:
def id_to_hash(customer_id):
    """Return a 16-bit integer hash of a customer id string"""
    return int(hashlib.md5(customer_id.encode('utf-8')).hexdigest(), 16)

In [3]:
members = pd.read_csv('/data/churn/members_v3.csv', nrows = 1)
members

transactions = pd.read_csv('/data/churn/all_trans.csv', nrows = 1)
transactions

logs = pd.read_csv('/data/churn/user_logs.csv', nrows = 1)
logs

train = pd.read_csv('/data/churn/all_train.csv', nrows = 1)
train

test = pd.read_csv('/data/churn/sample_submission_v2.csv', nrows = 1)
test

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911


,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0


,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,20150513,0,0,0,0,1,1,280.335


,msno,is_churn
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1


,msno,is_churn
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0


In [4]:
id_to_hash(members.loc[0, 'msno'])
id_to_hash(members.loc[0, 'msno']) % N_PARTITIONS

209512247756457468966515739358104959027

27

In [5]:
id_to_hash(transactions.loc[0, 'msno'])
id_to_hash(transactions.loc[0, 'msno']) % N_PARTITIONS

311407269432611323870693642675616983728

728

The following code creates `N_PARTITIONS` empty directories. The files within each directory will be named exactly the same so the directory name can be used to distinguish partitions.

In [6]:
import os

base_dir = '/data/churn/partitions/'

if not os.path.exists(base_dir + 'p999'):
    # Create a new directory for each partition
    for i in range(N_PARTITIONS):
        os.makedirs(base_dir + f'p{i}', exist_ok=False)
    
len(os.listdir(base_dir))

1000

## Create Files

In each partition there are 5 csv files. 

* `transactions.csv`
* `train.csv`
* `test.csv`
* `members.csv`
* `logs.csv`

The following code writes the header for each of the five files in each of the `N_PARTITION` partitions.

In [7]:
InteractiveShell.ast_node_interactivity = 'last_expr'
','.join(list(transactions.columns))

'msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel'

In [8]:
def create_blank_partitions():
    """Create blank files in each partition and write the file header"""
    # For each partition create the files with headers
    for i in range(N_PARTITIONS):
        directory = base_dir + f'p{i}/'
        # Create five files
        for file in ['transactions.csv', 'train.csv', 'test.csv', 'members.csv', 'logs.csv']:
            # Write file header as first line
            with open(directory + file, 'w') as f:
                if file == 'transactions.csv':
                    f.write(','.join(list(transactions.columns)))
                elif file == 'train.csv':
                    f.write(','.join(list(train.columns)))
                elif file == 'test.csv':
                    f.write(','.join(list(train.columns)))
                elif file == 'members.csv':
                    f.write(','.join(list(members.columns)))
                elif file == 'logs.csv':
                    f.write(','.join(list(logs.columns)))
                    
    return directory

In [9]:
directory = create_blank_partitions()
os.listdir(directory)

['logs.csv', 'members.csv', 'train.csv', 'test.csv', 'transactions.csv']

When it comes time to write the data, we'll open the existing files using append (`a`) option and add to whatever is there. At this point, each file just has a header.

In [10]:
pd.read_csv(directory + 'members.csv').head()

,msno,city,bd,gender,registered_via,registration_init_time


### Example of Writing a Row

For each file, one option to write the data to the partitions is to iterate over the rows one at a time. The process for handling a row is:

1. Convert the customer id to an integer by hashing
2. Convert the integer to a partition number by modulus dividing by the number of partitions
3. Append the row to the correct partition directory and file

Let's see how this works with a single row.

In [11]:
# Iterate through the dataframe one row at a time
for i, row in members.iterrows():
    # Find the partition by hashing the id
    partition = id_to_hash(row['msno']) % N_PARTITIONS
    # Open the file for appending
    with open(base_dir + f'p{partition}/members.csv', 'a') as f:
        # Write a newline and then the information
        f.write('\n')
        f.write(','.join([str(x) for x in row.values]))
    if i > 1:
        break

In [12]:
pd.read_csv(base_dir + f'p{partition}/members.csv')

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911


Everything looks like it went well with the first attempt. However, we might want to ask if iterating over the dataset one row at a time using `iterrows` is the quickest options.

# Performance of Different Methods

There are a number of different options to handle writing the data to the correct partition. To find out which method is the best, we'll try 4 approaches:

1. `df.iterrows()`: iterate through the dataframe one row at a time with rows represented as series
2. `df.itertuples()`: iterate through the dataframe one row at a time with rows represented as tuples
3. `df.apply()`: iterate through the data one row at a time using `apply`
4. `groupby(partition)` and save each group with `to_csv()`: iterate through the dataframe one partition at a time

The four approaches have different applicability and performance characteristics (see this [Stack Overflow](https://stackoverflow.com/questions/24870953/does-iterrows-have-performance-issues/24871316#24871316) answer). The way to find out which one is the quickest is to try them all. This isn't meant to represent all use cases, so your particular results may vary.

In [13]:
from timeit import default_timer as timer

### Member Information

We'll start off with the members. 

In [14]:
members = pd.read_csv('/data/churn/members_v3.csv')
members.shape

(6769473, 6)

## Iterrows

The first implementation to try is `iterrows`. This is fairly slow because Pandas packages the row as a Pandas series before iteration. However, it does allow us to access each value using conventional locating. 

In [15]:
start = timer()

for i, row in members.iterrows():
    # Find the partition number by hashing the id
    partition = id_to_hash(row['msno']) % N_PARTITIONS
    
    # Open file for appending
    with open(base_dir + f'p{partition}/members.csv', 'a') as f:
        # Write a new line and then data
        f.write('\n')
        f.write(','.join([str(x) for x in row.values]))
        
    if i % 10000 == 0:
        print(f'{100 * round(i / members.shape[0], 2)}% complete. {round(timer() - start)} seconds elapsed.', end = '\r')

end = timer()
print(f'Processing {i} lines took {round(end - start)} seconds using iterrows.')

Processing 6769472 lines took 557 seconds using iterrows.


In [16]:
pd.read_csv(base_dir + f'p{partition}/members.csv').head()

,msno,city,bd,gender,registered_via,registration_init_time
0,+zMKqjvsTvD7O0Fvntk3VXe4ovwvD4KYk6PJZ92Ky60=,1,0,NaN,9,20161227
1,3p2AY1tZAYa4LFcs0/plkuPv2hY9smh/xgcbKjtU9Dc=,5,26,male,3,20141109
2,BO0XUBzHeItkHI3N5g4uL08Ld1T/ZW/8GrbjBmT4s3w=,1,0,NaN,7,20161228
3,qy7PNK2EE4+x6xeIdqjFVw5FlmxnFKylv6LKqqGbSo4=,1,23,female,4,20170113
4,c36721uHBQyhoVko21J9rR44Fex2ul72a74k0M7IkiQ=,1,0,NaN,4,20170121


This approach works but is fairly slow.

## Itertuples

Itertuples should be faster than iterrows because Pandas packages the row as a tuple instead of as a series. The tradeoff is we need to be careful when accessing the elements of the series since we can't refer to them by name. For example, to make sure we are hashing the customer id (`msno`) we need to grab the second element of the tuple. 

In [17]:
_ = create_blank_partitions()

In [18]:
start = timer()

for i, tup in enumerate(members.itertuples()):
    
    # Find the partition number by hashing the id
    partition = id_to_hash(tup[1]) % N_PARTITIONS
    
    # Open file for appending
    with open(base_dir + f'p{partition}/members.csv', 'a') as f:
        # Write a new line and then data
        f.write('\n')
        f.write(','.join([str(x) for x in tup[1:]]))
        
    if i % 10000 == 0:
        print(f'{100 * round(i / members.shape[0], 2)}% complete. {round(timer() - start)} seconds elapsed.', end = '\r')

end = timer()
print(f'Processing {i} lines took {round(end - start)} seconds using itertuples.')

Processing 6769472 lines took 138 seconds using itertuples.


In [19]:
pd.read_csv(base_dir + f'p{partition}/members.csv').head()

,msno,city,bd,gender,registered_via,registration_init_time
0,+zMKqjvsTvD7O0Fvntk3VXe4ovwvD4KYk6PJZ92Ky60=,1,0,NaN,9,20161227
1,3p2AY1tZAYa4LFcs0/plkuPv2hY9smh/xgcbKjtU9Dc=,5,26,male,3,20141109
2,BO0XUBzHeItkHI3N5g4uL08Ld1T/ZW/8GrbjBmT4s3w=,1,0,NaN,7,20161228
3,qy7PNK2EE4+x6xeIdqjFVw5FlmxnFKylv6LKqqGbSo4=,1,23,female,4,20170113
4,c36721uHBQyhoVko21J9rR44Fex2ul72a74k0M7IkiQ=,1,0,NaN,4,20170121


This approach was much faster because Pandas does not have to convert each row into a Series which has more overhead than a tuple.

In [20]:
_ = create_blank_partitions()

## Apply

Another option would be to use the apply over the rows. To use `apply`, we write a small function that saves the row and then call apply to the dataframe using `axis = 1` which sends each row to the function. This also sends a row as a Series to the function, but in practice it seems to be much faster than `iterrows`.

In [21]:
def save_row(row, name):
    # Find the partition number by hashing the id
    partition = id_to_hash(row['msno']) % N_PARTITIONS
    
    # Open file for appending
    with open(base_dir + f'p{partition}/{name}.csv', 'a') as f:
        # Write a new line and then data
        f.write('\n')
        f.write(','.join([str(x) for x in row.values]))

In [22]:
from tqdm import tqdm_notebook
from tqdm import tqdm
tqdm.pandas()

start = timer()
members.progress_apply(save_row, axis = 1, name = 'members')
end = timer()

print(f'Processing {members.shape[0]} rows took {round(end - start)} seconds using apply.')

100%|██████████| 6769473/6769473 [04:41<00:00, 24048.87it/s]

Processing 6769473 rows took 282 seconds using apply.


In [23]:
pd.read_csv(base_dir + f'p{partition}/members.csv').head()

,msno,city,bd,gender,registered_via,registration_init_time
0,+zMKqjvsTvD7O0Fvntk3VXe4ovwvD4KYk6PJZ92Ky60=,1,0,NaN,9,20161227
1,3p2AY1tZAYa4LFcs0/plkuPv2hY9smh/xgcbKjtU9Dc=,5,26,male,3,20141109
2,BO0XUBzHeItkHI3N5g4uL08Ld1T/ZW/8GrbjBmT4s3w=,1,0,NaN,7,20161228
3,qy7PNK2EE4+x6xeIdqjFVw5FlmxnFKylv6LKqqGbSo4=,1,23,female,4,20170113
4,c36721uHBQyhoVko21J9rR44Fex2ul72a74k0M7IkiQ=,1,0,NaN,4,20170121


So `apply` is faster than `iterrows` but slower than `itertuples` (at least in this case). 

In [24]:
_ = create_blank_partitions()

## Groupby 

The final option we'll try is to group the data by the partition after converting all the customer ids into partition numbers in once operation.

1. Compute the partitions with the hashing function all at once
2. Groupby the partition
3. Write the grouped dataframe to the correct partition directory and file

To find out the quickest way to convert all the customer ids to an integer, we can compare `map` and `apply`. 

In [25]:
%%timeit -n 1 -r 3
members['msno'].map(id_to_hash) % 1000

8.02 s ± 82.3 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [26]:
%%timeit -n 1 -r 3
members['msno'].apply(id_to_hash) % 1000

7.87 s ± 9.26 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


It looks like `apply` is slightly faster although the difference is not significant. We'll convert all the customer ids to the partitions once so this is not a large time cost. Before we go on, we should make sure that the hashing and conversion to partition operation is creating partitions close to the same size.

In [27]:
members['partition'] = members['msno'].apply(id_to_hash) % 1000
members['partition'].value_counts().head()

689    7027
842    7012
844    6992
660    6986
91     6983
Name: partition, dtype: int64

In [28]:
members['partition'].value_counts().describe()

count    1000.000000
mean     6769.473000
std        81.213231
min      6494.000000
25%      6716.000000
50%      6773.000000
75%      6825.000000
max      7027.000000
Name: partition, dtype: float64

It looks like the number of members in each partition is fairly constant. We can check another dataset to make sure.

In [29]:
transactions = pd.read_csv('/data/churn/all_trans.csv')
transactions['partition'] = transactions['msno'].apply(id_to_hash) % 1000
transactions['partition'].value_counts().describe()

count     1000.00000
mean     22978.75500
std        652.76208
min      20893.00000
25%      22532.75000
50%      22975.00000
75%      23420.25000
max      25353.00000
Name: partition, dtype: float64

The following cell runs the group by approach to partitioning the data. The biggest thing to watch out for is making sure we are appending to the file each time (open with `a`). When we write to a csv with `to_csv`, we can pass in an already open file. We also don't write a header since we already created the headers in every file, and do not write the index.

In [30]:
start = timer()
members['partition'] = members['msno'].apply(id_to_hash) % N_PARTITIONS

# Iteration through grouped partitions
for partition, grouped in members.groupby('partition'):
    
    grouped = grouped.drop(columns = 'partition')
    # Open file for appending
    with open(base_dir + f'p{partition}/members.csv', 'a') as f:
        f.write('\n')
        grouped.to_csv(f, header = False, index = False)
        
end = timer()
print(f'Processing {members.shape[0]} rows took {round(end - start)} seconds using groupby.')

Processing 6769473 rows took 41 seconds using groupby.


In [31]:
pd.read_csv(base_dir + f'p{partition}/members.csv').head()

,msno,city,bd,gender,registered_via,registration_init_time
0,bpIibSSY6wymQbGaQOR9q6dcWKg7lUfw3Y+LttzAQNQ=,1,0,NaN,4,20170104
1,HZwqy9brMyBDuFVpXlAqli8yoAixLc1rA0ExAZYZR50=,1,0,male,3,20130220
2,kW0/xDZUihRKFMa3ti+vq3fF/O2li5aYpY+szvzg0ko=,1,0,NaN,3,20130227
3,yCUq5TNkbcJF0inE45ICYI//gZ+FzPmwSZWmFie4nk8=,6,31,female,9,20130305
4,VsM62mNuBRPH2YZSZKaRlD0IQsqoJa55aKxukV84oY4=,1,0,female,3,20150213


The group by approach to partitioning the data is by far the fastest method. We'll put this into a function to use with all the datasets.

## Reusable Hashing Dataframe Function

To make the process reusable, we'll write a function that does this for us. It will take in a dataframe, a name for the file to save the data to, and an optional progress argument. The function will map the customer id (`msno`) to a partition number using the hash modulo the number of partitions, group the dataframe by the partition, and write the grouped dataframe to the appropriate directory. 

In [32]:
members = pd.read_csv('/data/churn/members_v3.csv', nrows = 1)
transactions = pd.read_csv('/data/churn/all_trans.csv', nrows = 1)
logs = pd.read_csv('/data/churn/user_logs.csv', nrows = 1)
train = pd.read_csv('/data/churn/all_train.csv', nrows = 1)
test = pd.read_csv('/data/churn/sample_submission_v2.csv', nrows = 1)

_ = create_blank_partitions()

In [33]:
pd.read_csv('/data/churn/partitions/p999/members.csv').head()

,msno,city,bd,gender,registered_via,registration_init_time


In [34]:
def partition_by_hashing(df, name, progress = None):
    """Partition a dataframe into N_PARTITIONS by hashing the id.
    
    Params
    --------
        df (pandas dataframe): dataframe for partition. Must have 'msno' column.
        name (str): name of dataframe. Used for saving the row data.
        progress (int, optional): number of rows to be processed before displaying information.
                                  Defaults to None
                                  
    Returns:
    --------
        Nothing returned. Dataframe is saved one line at a time as csv files to the N_PARTITIONS 
    """
    start = timer()
    
    # Map the customer id to a partition number
    df['partition'] = df['msno'].apply(id_to_hash) % N_PARTITIONS
    
    # Iterate through one row at a time
    for partition, grouped in df.groupby('partition'):
        
        # Don't need to save the partition column
        grouped = grouped.drop(columns = 'partition')
        
        # Open file for appending
        with open(base_dir + f'p{partition}/{name}.csv', 'a') as f:
            # Write a new line and then data
            f.write('\n')
            grouped.to_csv(f, header = False, index = False)
            
        # Record progress every `progress` steps
        if progress is not None:
            if partition % progress == 0:
                print(f'{100 * round(partition / N_PARTITIONS, 2)}% complete. {round(timer() - start)} seconds elapsed.', end = '\r')
    
    end = timer()
    if progress is not None:
        print(f'\n{df.shape[0]} rows processed in {round(end - start)} seconds.')

In [35]:
members = pd.read_csv('/data/churn/members_v3.csv')
partition_by_hashing(members, name = 'members', progress = 10)

99.0% complete. 41 seconds elapsed.onds elapsed..
6769473 rows processed in 41 seconds.


In [36]:
pd.read_csv(base_dir + f'p{partition}/members.csv').head()

,msno,city,bd,gender,registered_via,registration_init_time
0,bpIibSSY6wymQbGaQOR9q6dcWKg7lUfw3Y+LttzAQNQ=,1,0,NaN,4,20170104
1,HZwqy9brMyBDuFVpXlAqli8yoAixLc1rA0ExAZYZR50=,1,0,male,3,20130220
2,kW0/xDZUihRKFMa3ti+vq3fF/O2li5aYpY+szvzg0ko=,1,0,NaN,3,20130227
3,yCUq5TNkbcJF0inE45ICYI//gZ+FzPmwSZWmFie4nk8=,6,31,female,9,20130305
4,VsM62mNuBRPH2YZSZKaRlD0IQsqoJa55aKxukV84oY4=,1,0,female,3,20150213


## Training Data

Now we can use this function to partition the training data.

In [37]:
train = pd.read_csv('/data/churn/all_train.csv')
partition_by_hashing(train, name = 'train', progress = 10)

99.0% complete. 8 seconds elapsed.onds elapsed..
1963891 rows processed in 8 seconds.


In [38]:
pd.read_csv(base_dir + f'p{partition}/train.csv').head()

,msno,is_churn
0,Ad9xf2W6ID3zguduv1lKdla80V/iT2cFWbDxIcEMQOs=,1
1,nL5mKfbpD9mjLJiIbOa1MsrHXKlmC4Nt5S3ieKtqUq8=,1
2,TmSYOz2xCdk5j1hmUP72/FQdkU3kpoqEl9RDc9UBtf8=,1
3,NNYVPCEq8Pk2QYIGSBSIiO+XcX/Sqa2TG5+szr4DMuk=,1
4,bhzedCaoyawwHOssUE6IXI1BP4I0/4nDC1H6CtXcIK4=,1


## Testing Data

The nice thing about a function is we can keep applying it, changing only the arguments! 

In [39]:
test = pd.read_csv('/data/churn/sample_submission_v2.csv')
partition_by_hashing(test, name = 'test', progress = 10)
pd.read_csv(base_dir + f'p{partition}/test.csv').head()

99.0% complete. 4 seconds elapsed.onds elapsed..
907471 rows processed in 4 seconds.


,msno,is_churn
0,bFTbnI7GC8TZJ2m9dfLF0bkFIEsy/0ERZzmFFekPWpY=,0
1,SvdAOG3xp9glZIopNrlnOOiOLHdmnpHfwRGnNUpOrmo=,0
2,GG+L3Jv6naoL8JjQYPIem7ISmtoHcMX453sqTSFcl2Q=,0
3,OdT/fU9BKs+KrLeypHtoeQut97PBNWlDvjnMCUsJVzY=,0
4,f0SvlQcRZBgZbOaoPBkj57xo81+GnlluK1rLyy0PQnc=,0


## Transactional Data

The second to last dataset is the customer transactions.

In [40]:
transactions = pd.read_csv('/data/churn/all_trans.csv')
partition_by_hashing(transactions, name = 'transactions', progress = 10)
pd.read_csv(base_dir + f'p{partition}/transactions.csv').head()

99.0% complete. 199 seconds elapsed.onds elapsed.
22978755 rows processed in 201 seconds.


,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,igTF6Ef1Y1chfDlEjV+59Hgp4mfh8ZNVXF/vlv1TvhY=,41,30,129,129,1,20160418,20160518,0
1,wlwPlei0VKJ6AF07YrL04gpbebUPitqtnqa3CnL4tmI=,41,30,149,0,1,20150107,20160106,0
2,m40NTt7XHbAVwCI4y6TYH0XBJdPeVVpBUSKT9NXP7xE=,39,30,149,149,1,20161231,20170210,0
3,wQ8y0ZtCUniH1rWDKyltgP+nfxDV9hAvjcFBa7l9uAU=,41,30,99,99,1,20160731,20160831,0
4,CZ87lCEDKmbk7EXCy6Kh0zopNJxoEBTfWq9Eq9Az+28=,32,195,894,894,0,20160726,20170209,0


## User Log Data

The final dataset cannot be pass directly into the function because of the size which won't even let us read the entire file into memory. Instead, we can read it in a chunk at a time using Pandas and apply the function to each chunk. There are actually two log files of significantly different sizes, but we'll use the same chunking approach for each.

In [41]:
print(os.stat('/data/churn/user_logs.csv').st_size / 1e9)
print(os.stat('/data/churn/user_logs_v2.csv').st_size / 1e9)

30.514081415
1.431465728


The second user logs can be processed in the previous manner because it can be read completely into memory, but we'll go ahead and apply the chunking method. The chunksize refers to the number of rows read in at a time. Using Pandas `read_csv` and specifying the `chunksize`, we can then iterate over the file one chunk at a time. 

In [42]:
chunksize = 1e6
start = timer()

for chunk in pd.read_csv('/data/churn/user_logs_v2.csv', chunksize = chunksize):
    partition_by_hashing(chunk, name = 'logs', progress = None)
    
    if (i + 1) % 10 == 0:
        print(f'{i * chunksize} rows processed.', end = '\r')

end = timer()
print(f'\nOverall time: {round(end - start)} seconds.')


Overall time: 200 seconds.


In [43]:
pd.read_csv(base_dir + f'p{partition}/logs.csv').head()

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,555bSTll4Rzaz1vBg/VfGfaXKEE8S74EAdKPOCkBqpg=,20170301,2,0,0,1,25,26,6306.242
1,KORJwjTctoWENM9oM2Rrl432wxllmC1RvP5p84PUFI8=,20170316,6,1,2,3,36,34,10718.255
2,+GdZIQJdsQSeRKyu/GONhgWzK4R8Ufm59RpzwAj4OCE=,20170310,5,1,0,0,21,21,4948.548
3,9yZc5dve4sg96RQT1FyTPSmcDWrIVu+qsI7W7oOmFvE=,20170306,0,0,1,0,28,1,3436.411
4,Rq9NMOw9RAbN3qUIG1MUU+vL0lqhAJUAsZiW0dsm/Lg=,20170322,9,7,4,2,48,67,13534.787


The final dataset has about 400 million rows. We can read it in 10 million rows at a time. 

In [44]:
chunksize = 1e7

start = timer()

for i, chunk in enumerate(pd.read_csv('/data/churn/user_logs.csv', chunksize = chunksize)):
    partition_by_hashing(chunk, name = 'logs', progress = None)
    
    if (i + 1) % 10 == 0:
        print(f'{i * chunksize} rows processed.', end = '\r')
    
end = timer()
print(f'\nOverall time: {round(end - start)} seconds.')

390000000.0 rows processed.
Overall time: 3819 seconds.


With the grouping approach, we were able to get 30 GB of data partitioned in less than 2 hours! Dealing with large datasets can be tough, but breaking down the problem makes it much more manageable. 

In [45]:
pd.read_csv(base_dir + f'p{partition}/logs.csv').tail()

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
402013,fBNdslCoBSVgRLCTT/1wz2u5BJE0D4TuF+8g0+IPOLQ=,20160509,0,0,0,0,15,15,4187.548
402014,fBNdslCoBSVgRLCTT/1wz2u5BJE0D4TuF+8g0+IPOLQ=,20160703,1,0,0,0,4,4,924.083
402015,fBNdslCoBSVgRLCTT/1wz2u5BJE0D4TuF+8g0+IPOLQ=,20161004,2,0,0,0,24,26,6031.879
402016,fBNdslCoBSVgRLCTT/1wz2u5BJE0D4TuF+8g0+IPOLQ=,20170101,0,3,2,0,1,5,790.686
402017,xVJ8UUCOfXv5dXwxjFP5ffOHP+kgRRZbM7Mf5hJe0AE=,20150126,2,0,0,0,1,3,321.985


# Conclusions

In this notebook, we implemented a partitioning of a dataset that would normally be too large to fit in memory. After trying several options, we eventually decided on the fastest process which was:

1. Map customer ids to a partition using a hashing function.
2. Compute the integer hash and then divide by the number of partitions 
3. Group the dataframe by the partition and write each partition to appropriate directory and file
4. For large files that cannot fit all in memory, read in via chunking and send each chunk through the partitioning function

Now we can work on an individual partition to develop an automated feature engineering pipeline. After the pipeline has been developed, we can use a framework such as Spark or Dask to run the partitions through the pipeline in parallel. This will speed up the overall feature engineering process and allow us to scale to larger datasets.

## Next Steps

To implement a machine learning solution, we need to take several steps outlined in the following process:

1. Prediction Engineering: define a business need and translate into a machine learning problem. Create a set of labeled historical examples (called label times) that can be used to build features for each label.
2. Use the label times to build features for each label by filtering the data to times before the cutoff time. This procedure can be rapidly completed using automated feature engineering.
3. Train a machine learning algorithm to predict the labels from the features. Once the model has been optimized, use it to make predictions on new data.

With the data in partitions, the first two steps can be done rapidly in parallel. The first step is implemented in the `Prediction Engineering` notebook.